In [1]:
import matplotlib
%matplotlib tk
%autosave 180
%load_ext autoreload
%autoreload 2

#
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# 
import matplotlib.cm as cm

# 
import numpy as np
import os
from tqdm import trange
import parmap
import glob
from sklearn.decomposition import PCA
#import umap
import seaborn as sns
import pandas as pd

import pickle
# 
from tqdm import tqdm

import sklearn.experimental
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# 
from scipy.io import loadmat
import scipy

# 

Autosaving every 180 seconds


In [2]:
######################################
############ FILTER DATA #############
######################################
import scipy
import scipy.ndimage

class CentreBody():
    
    def __init__(self):
         
        # 
        self.node_names = ['nose',          # 0
                          'lefteye',       # 1
                          'righteye',      # 2
                          'leftear',       # 3
                          'rightear',      # 4
                          'spine1',        # 5
                          'spine2',        # 6
                          'spine3',        # 7
                          'spine4',        # 8
                          'spine5',        # 9
                          'tail1',         # 10
                          'tail2',         # 11
                          'tail3',         # 12
                          'tail4']         # 13

        self.feature_ids = np.array([0,5,6,7,8,9])       
        
        
    def get_fnames(self):

        self.fnames = glob.glob(self.root_dir+"/*_compressed.npy")

    
    def filter_data(self):
        
        print ("  ... median filtering ...")

            
        if self.parallel:
            parmap.map(self.filter_data1, self.fnames,
                      pm_processes=8,
                      pm_pbar=True)
        else:
            for fname in fnames:
                pass

    def filter_data1(self, fname):    

        fname_out = fname.replace('.npy','_median_filtered.npy')
        if os.path.exists(fname_out)==False:
            data = np.load(fname)

            data_filtered = data.copy()
            # loop over animals
            for a in range(data.shape[1]):
                # loop over feature
                for f in range(data.shape[2]):
                    # loop over each x,y
                    for l in range(data.shape[3]):
                        data_filtered[:,a,f,l] = filter_data2(data[:,a,f,l])

            np.save(fname_out, data_filtered)


    def filter_data2(self, x, width=25):

        # replace data with previous value until all nans are replacesd
        for k in range(1000):
            idx = np.where(np.isnan(x))[0]
            if idx.shape[0]==0:
                break

            if idx[0]==0:
                idx=idx[1:]
            x[idx] = x[idx-1]

        
        x = scipy.ndimage.median_filter(x, width=width)

        return x
    

    def reject_outliers2(self, x,y,
                        max_dist_pairwise,
                        max_dist_all=100):  # number of deviations away

        ''' Function returns indexes for which [x,y] array points are close to at least 2 other points

            Goal 1: to generate very clean data which has 6 body features well connected for downstream analysis and imputation

            Goal 2: to remove outliers and leave small clusters of feautres only
            
        '''

        # method 2: explicitly reject points that are > max distance from nearest 2 points
        temp = np.vstack((x,y))
        dists = scipy.spatial.distance.cdist(temp.T, temp.T)

        # first check points inside the array to ensure they have 2 close neighbours
        # if they don't, remove them so other points can't be connected to them.
        idx_far = []
        for k in range(1,temp.shape[1]-1,1):
            #idx = np.where(dists[k]<=max_dist_pairwise)[0]
            temp = dists[k]
            if np.abs(temp[k]-temp[k-1])>max_dist_pairwise or np.abs(temp[k]-temp[k+1])>max_dist_pairwise:
                idx_far.append(k)
                dists[:,k]= 1E3

        # check start and end points to ensure they have nearby val
        if np.abs(dists[0,1])>max_dist_pairwise:
            idx_far.append(0)
            #print (dists[0], 'excluded ', 0)

        if np.abs(dists[dists.shape[1]-1,dists.shape[1]-2])>max_dist_pairwise:
            idx_far.append(dists.shape[1]-1)
            #print (dists[0], 'excluded ', dists.shape[1]-1)


        x[idx_far] = np.nan
        y[idx_far] = np.nan

        return x, y


    def reject_outliers1(self, fname, feature_ids, max_dist):
        
        fname2 = fname.replace('.npy','_median_filtered.npy')
        
        fname_out = fname2.replace('.npy','_outliers.npy')
        
        if os.path.exists(fname_out)==False:
            
            data = np.load(fname2)
            for f in range(0,data.shape[0],1):

                for k in range(data.shape[1]):
                    # 
                    x = data[f,k,feature_ids,0]
                    y = data[f,k,feature_ids,1]

                    x, y = self.reject_outliers2(x,y, max_dist)

                    data[f,k,feature_ids,0] = x
                    data[f,k,feature_ids,1] = y

            np.save(fname_out, data)

    def reject_outliers(self, max_dist=40):
        
        print ("  ... rejecting outliers....")
        
        self.fnames = glob.glob(self.root_dir+"/*_compressed.npy")

        if self.parallel:
            parmap.map(self.reject_outliers1, self.fnames, 
                       self.feature_ids,
                       max_dist,
                       pm_processes=8,
                       pm_pbar=True)
        else:
            for fname in fnames:
                pass


    def centre_and_align2(self, data, frame, centre_pt=0):

        if True:
            # centre the data on the nose
            data[:,0] -= data[centre_pt,0]
            data[:,1] -= data[centre_pt,1]

            # get angle between +x axis and head location (i.e. 2nd position)
            t = -np.arctan2(*data[1].T[::-1])-np.pi/2

            # get rotation
            rotmat = np.array([[np.cos(t), -np.sin(t)], 
                               [np.sin(t),  np.cos(t)]])

            # Apply rotation to each row of m
            m2 = (rotmat @ data.T).T

            return m2

    
    def centre_and_align(self):
        
        print ("  ... center and aligning ...")

        self.fnames = glob.glob(self.root_dir+"/*_compressed.npy")

        if self.parallel:
            parmap.map(self.centre_and_align1, self.fnames, 
                       self.feature_ids,
                       pm_processes=8,
                       pm_pbar=True)
        else:
            for fname in fnames:
                pass
            

    def centre_and_align1(self, fname,
                         feature_ids):

        fname2 = fname.replace('.npy','_median_filtered_outliers.npy')
        
        fname_out = fname2.replace('.npy','_centre_aligned.npy')
        #fname_out_good_only = fname2.replace('.npy','_centre_aligned.npy')
        
        if os.path.exists(fname_out)==False:
            
            data = np.load(fname2)

            # 
            centre_pt = 0

            features_full = np.zeros((data.shape[0],data.shape[1],feature_ids.shape[0],2), 
                                          'float32')+np.nan

            features_array = []
            for k in range(4):
                features_array.append([])

            for f in range(0,data.shape[0],1):

                # loop over each animal
                for k in range(data.shape[1]):

                    x = data[f,k,feature_ids,0]
                    y = data[f,k,feature_ids,1]

                    idx = np.where(np.isnan(x))[0]
                    if idx.shape[0]==0:
                        #print (f, k, x.shape)

                        locs = np.vstack((x,y)).T

                        # centre and align data
                        locs_pca = self.centre_and_align2(locs,f,centre_pt)

                        if locs_pca is not None:
                            idx = np.where(np.isnan(locs_pca))[0]

                            if idx.shape[0]>0:
                                continue
                                
                            features_full[f,k] = locs_pca
                            features_array[k].append(locs_pca)

            np.save(fname_out, features_full)

    def load_processed_data(self, data_type=1, remove_nans=True):
        
        ''' data_type can be 3 types:
            0: filtered, outlier cleaned, centred and rotated on the nose/head
            1: filtered, outlier cleaned <- currently used
            2: filtered only <- very noisy data with many outlier vals
        
            remove_nans - cleans up the data so only full body axis data is used
            True: loading clean data for model training
            False: loading all data for prediction
        
        '''
            
        March16_file_order = [

        '2020-3-16_11-56-56-704655',  # day time starts correct day
        '2020-3-16_12-57-12-418305',
        '2020-3-16_01-57-27-327194',
        '2020-3-16_02-57-41-995158',
        '2020-3-16_03-57-56-902379',
        '2020-3-16_04-58-11-998956',
        '2020-3-16_05-58-27-193818',
        '2020-3-16_06-58-43-678014',
        '2020-3-16_07-59-00-362242',
        '2020-3-16_08-59-17-534732',
        '2020-3-16_09-59-34-731308',
        '2020-3-16_10-59-50-448686',

        '2020-3-16_12-54-07-193951',  # night time of previous day though
        '2020-3-16_01-54-23-358257',
        '2020-3-16_02-54-39-170978',
        '2020-3-16_03-54-54-231226',
        '2020-3-16_04-55-09-841582',
        '2020-3-16_05-55-25-305681',
        '2020-3-16_06-55-40-714236',
        '2020-3-16_07-55-55-775234',
        '2020-3-16_08-56-11-096689',
        '2020-3-16_09-56-26-362091',
        '2020-3-16_10-56-41-406701',
        ]

        # stack the postures for each animal for the day 
        features_array = []
        for k in range(4):
            features_array.append([])

        for file in tqdm(March16_file_order):

            # this data is filtered, outlier triaged + centred to nose and aligned to face up
            if data_type==0:
                fname = glob.glob(os.path.join(cb.root_dir,file+"*_centre_aligned.npy").replace("-","_"))[0]
                d3 = np.load(fname)

            # this data is filtered and outlier triaged
            if data_type==1:
                fname = glob.glob(os.path.join(cb.root_dir,file+"*_median_filtered_outliers.npy").replace("-","_"))[0]    

                d3 = np.load(fname)
                d3 = d3[:,:,self.feature_ids]  

            # this data is not outlier traiged; - very poor results/bad to work with
            if data_type==2:
                fname = glob.glob(os.path.join(cb.root_dir,file+"*_median_filtered.npy").replace("-","_"))[0]    

                d3 = np.load(fname)
                d3 = d3[:,:,self.feature_ids]       

            #print ("d3: ", d3.shape)
            # loop over animals and keep only complete data (i.e. 6 pts)
            for k in range(4):
                
                # find nans and delete any frame that is missing even a single value for that animal
                if remove_nans:

                    idx = np.where(np.isnan(d3[:,k]))
                    ids, counts = np.unique(idx[0], return_counts=True)
#                     print (k, "Coutns: ", counts.shape, counts)
#                     print (k, "ids: ", ids.shape, " idx: ", idx[0].shape, idx[0])
                    
                    idx_all = np.arange(d3.shape[0])
                    idx_good = np.delete(idx_all, ids)

                    temp = d3[idx_good,k]
                
                    features_array[k].append(temp)
                else:
                    features_array[k].append(d3[:,k])

                    
        # return stacked feature array
        self.features_array = features_array

    
cb = CentreBody()
cb.parallel = True
cb.root_dir = '/media/cat/1TB/dan/cohort1/slp/'
cb.get_fnames()

# median filter data
cb.filter_data()

# 
cb.reject_outliers()

#
cb.centre_and_align()

########################################
# data_type:  0 - filtered, outlier triaged + centred to nose and aligned to face up
#             1 - filtered, outlier triaged 
#             2 - filtered
cb.load_processed_data(data_type=0, 
                       remove_nans=True)

print (" DATA SIZES: ")
print (" female: ", np.vstack(cb.features_array[0]).shape)
print (" male: ", np.vstack(cb.features_array[1]).shape)
print (" pup1: ", np.vstack(cb.features_array[2]).shape)
print (" pup2: ", np.vstack(cb.features_array[3]).shape)

# 




  ... median filtering ...


100%|██████████| 23/23 [00:00<00:00, 21624.97it/s]

  ... rejecting outliers....



100%|██████████| 23/23 [00:00<00:00, 12110.09it/s]


  ... center and aligning ...


100%|██████████| 23/23 [00:01<00:00, 12.90it/s]

 DATA SIZES: 
 female:  (218641, 6, 2)
 male:  (94647, 6, 2)
 pup1:  (132861, 6, 2)
 pup2:  (176982, 6, 2)


In [154]:
####################################################
################ IMPUTATION STEP ##################
####################################################

# 
class Impute():
    
    def __init__(self, root_dir):
        
        self.root_dir = root_dir

    #   
    def evaluate_imputation_error(self,features_array, animal_id, res, idx_train, idx_test):

        temp = np.vstack(features_array[animal_id])
        temp = temp-temp[:,0][:,None]

        fig=plt.figure()
        ax=plt.subplot()

        diff = np.abs(temp[idx_test]-res)
        #print ("diff: ", diff.shape)

        errors = []
        for k in range(diff.shape[1]):
            errors.append([])

        for k in range(diff.shape[0]):
            for p in range(diff.shape[1]):
                temp = diff[k,p]
                #print (temp)
                tdiff = np.linalg.norm(temp)
                if tdiff>0:
                    errors[p].append(tdiff)

        t =[]
        for k in range(len(errors)):
            temp = errors[k]
            pad = np.zeros(100000-len(errors[k]),'float32')+np.nan
            temp = np.concatenate((temp, pad))
            t.append(temp)

        data = np.array(t).T
        #print (data.shape)
        columns = ['nose','spine1','spine2', 'spine3', 'spine4', 'spine5']
        #columns = ['errors']
        df = pd.DataFrame(data, columns = columns)

        #print ("DF: ", df)

        # plot
        ax=plt.subplot(2,1,1)
        sns.violinplot(data=df) #x=df['spine2'])
        plt.ylim(bottom=0)
        plt.ylabel(" pixel error")

        ax=plt.subplot(2,1,2)
        plt.title("Zoom",fontsize=20)
        sns.violinplot(data=df) #x=df['spine2'])
        plt.ylabel(" pixel error")
        plt.ylim(0,50)

         
    # 
    def predict_imputation_ground_truth(self, 
                                        drops=None, 
                                        idx_test=None, 
                                        n_drops = 3):

        '''  Function used to test the prediction between the ground truth complete body axis and dropout data
        
            Need to set the 'remove_nans' flag about to True to get clean model data 
            
            Need to set the idx_test to ~90% of data and run validation on the 10% holdout.
        
        '''
            
        # 

        
        if self.fname_dropout is not None:
            X_test = np.loadtxt(self.fname_dropout)
            print (X_test.shape)
            
            idx_drop = []
            
        # GENERATE RANDOM DROPS
        else:
            # 
            print ("LOADING head-egocentric clean data")
            self.cb.load_processed_data(data_type=0, remove_nans=True)
            temp = np.vstack(self.cb.features_array[self.animal_id])
            print (temp.shape)

            print ("Centering to head only <----------- NEEDS UPDATE FOR OTHER EGOCENTRIC DATA")
            temp = temp-temp[:,0][:,None]

            X_all = temp.reshape(temp.shape[0],-1)

            # select frames to predict
            if idx_test is not None:
                X_test = X_all[idx_test]
            else:
                X_test = X_all

            # do drop outs in the test set:
            X_test = X_test.reshape(-1,6,2)
            idx_drop = np.zeros((n_drops, X_test.shape[0]),'int32')
            for k in range(idx_drop.shape[1]):#n_drops):
                if drops is None:
                    idx_drop[:,k] = np.random.choice(np.arange(6),n_drops,replace=False)
                else:
                    idx_drop[:,k] = drops #np.random.choice(np.arange(6),n_drops,replace=False)


            print ("idx drop: ", idx_drop[0].shape)
            for k in range(len(idx_drop)):
                for p in range(idx_drop[k].shape[0]):
                    X_test[p,idx_drop[k][p]]=np.nan

            # 
            X_test = X_test.reshape(X_test.shape[0],-1)

        
        # TEST STEP
        #res = imp.transform(X_test).reshape(-1,6,2)
        print ("... predting data ...")
        #res = self.models[self.animal_id][0].transform(X_test).reshape(-1,6,2)
        res = self.model.transform(X_test).reshape(-1,6,2)
        #print ("Res: ", res.shape)

        self.res = res
        self.drop = idx_drop
        
        return res, idx_drop

    #
    def load_models(self):
        
        body_centre = 0
        # 
        
        #fname_in = self.root_dir+"imputation_animal_id"+str(self.animal_id)+"_body_centre"+str(p)+".pckl"
        fname_in = os.path.join(self.root_dir,
                                "model_type"+str(self.model_type)+
                                "_imputation_animal_id"+str(self.animal_id)+
                                "_body_centre"+str(body_centre)+".pckl")
        
        with open(fname_in, "rb") as f:
            model = pickle.load(f)

        self.model = model

    #
    def calculate_missing_features(self):
        
        ''' Function used to grab statistisc of missing data
        
        '''
        
        animal_ids = np.arange(4)

        idxc_array = []
        for animal_id in tqdm(animal_ids):
            I.animal_id = animal_id
            idxc_array.append([])
            for max_drops in range(7):

                # 
                temp = np.vstack(self.cb.features_array[self.animal_id])

                # find which frames have that feature intact
                idx = np.where(np.isnan(temp))
                ids, counts = np.unique(idx[0], return_counts=True)

                # these are the frames where at least max_drops worth of frames are present
                idxc = np.where(counts<=max_drops*2)[0]

                # these are frames where all 6 features are available.
                idx_all = np.arange(temp.shape[0])
                idx_good = np.delete(idx_all, ids)
                
                idxc_array[animal_id].append(idxc.shape[0] + idx_good.shape[0])
        
        self.idxc_array = idxc_array
        print (self.idxc_array)
        ax=plt.subplot()

        t = np.arange(6,-1,-1)
        for k in range(len(self.idxc_array)):
            plt.plot(t, np.array(self.idxc_array[k])/2069710., label=" animal: "+str(k))
            #plt.plot(t, I.idxc_array[k], label=" animal: "+str(k))

        plt.legend()
        plt.ylim(0,1)
        plt.xlabel("min # of present features ", fontsize=20)
        ax.ticklabel_format(useOffset=False)
        ax.ticklabel_format(style='plain')
        ax.set_ylabel("% of total # of frames in 23hr period",
                     fontsize=20)
        plt.plot([3,3],[0,1],'--')
        #plt.xlim(-0.1,6.1)
        plt.show()
        
    #
    
    
    
    def plot_imputation_results(self, features_array, animal_id, idx_test, res, idx_drop):

        # 
        labels = ['n','s1','s2','s3','s4','s5']

        # grab the selected data:
        temp = np.vstack(features_array[animal_id])
        temp = temp-temp[:,0][:,None]

        #print ("temp: ", temp.shape)
        fig = plt.figure()
        ax = plt.axes()
        shift = 0
        for k in range(10):
            plt.subplot(2,5,k+1)

            ############ PLOT GROUND TRUTH ##############
            id2 = np.random.choice(idx_test,1)[0]
            #print (id2)

            #print (temp[id2].shape)
            print (temp[id2,:,0])
            plt.scatter(temp[id2,:,0],
                        temp[id2,:,1],
                        c='blue',
                        s=np.arange(1,7,1)[::-1]*20,
                        alpha=.7,
                        edgecolor='black', label='truth')

            #
            id3 = np.where(idx_test==id2)[0]
            #print (id3)

            ############ PLOT IMPUTED LOCS ##############
            plt.scatter(res[id3,:,0]+shift,
                        res[id3,:,1],
                        c='red',
                        s=np.arange(1,7,1)[::-1]*20,
                        alpha=.7,
                        edgecolor='black', label='imputed')

            if True: #k==0:
                for p in range(6):
    #                 plt.text(res[id3,p,0],
    #                          res[id3,p,1],labels[p])
                    plt.text(temp[id2,p,0],
                             temp[id2,p,1],labels[p])

            # draw lines
            for p in range(len(idx_drop)):
                #print ("connectgin: ", p, idx_drop[p][id3])
                #print ("idx_drop full: ", np.array(idx_drop).shape)
                plt.plot([temp[id2,idx_drop[p][id3],0], res[id3,idx_drop[p][id3],0]+shift],
                         [temp[id2,idx_drop[p][id3],1], res[id3,idx_drop[p][id3],1]],
                         '--',c='black')

            if k==0:
                plt.legend(fontsize=8)

            plt.title("frame id: "+str(id2) + "\ndrops: "+str(np.array(idx_drop)[:,id3]),fontsize=8)

            x1 = (np.max(np.abs(temp[id2,:,0])), 
                                 np.max(np.abs(res[id3,:,0])),
                                 np.max(np.abs(temp[id2,:,1])), 
                                 np.max(np.abs(res[id3,:,1])))
            #print ("x1: ", x1)

            max_ = np.max(x1)*1.2
            #plt.xlim(-max_, max_+shift)
            plt.xlim(-200, 200)
            plt.ylim(-200,200)
            #plt.ylim(-max_,10)
            ax.set_aspect('equal', 'datalim')
            #print ('')

        plt.show()

        
    def predict_imputation_new_data(self, max_drops=3):
        
        ''' Function used to impute missing data
        
        '''
        
        # NED
        
        print ("NEED TO LOAD NOnNAN DATA FIRST")
        self.cb.load_processed_data(data_type=0, remove_nans=False)
        
        # 
        all_locs = np.vstack(self.cb.features_array[self.animal_id])

        # find which frames have that feature intact
        idx = np.where(np.isnan(all_locs))
        ids, counts = np.unique(idx[0], return_counts=True)
        
        #idx_all = np.arange(all_locs.shape[0])
        #idx_good = np.delete(idx_all, ids)

        # find frame Ids where at most max_drops of features are missing
        idxc = np.where(counts<=max_drops*2)[0]
        
        # 
        X_all = all_locs[idxc]
        print ("X_all: ", X_all.shape)
        
        # loop over each feature
        res = np.zeros((6,X_all.shape[0],
                              X_all.shape[1],
                              X_all.shape[2]),'float32')+np.nan
        for k in range(6):
            
            # find frames where current feature is present so we can apply model to it
            idx_feature = np.where(np.isnan(X_all[:,k,0])==False)[0]
            
            X_test = X_all[idx_feature]

            # load model
            temp_model = self.models[self.animal_id][k]
            #print (k, temp_model)

            # centre the data on the selected feature
            offsets = X_test[:,k]
            X_test = X_test-offsets[:,None]
            
            #print ("XTES: ", X_test.shape)

            # format the data
            X_test = X_test.reshape(X_test.shape[0],-1)
        
            # run mprediction
            r = temp_model.transform(X_test).reshape(-1,6,2)
            print ("Result: ", r.shape)
            
            # add offset back in
            r = r + offsets[:,None]
            #
            res[k,idx_feature] = r

        
        # average over all models 
        print (" res: ", res.shape)
        #res_ave = np.nanmean(res,axis=0)
        
        # use only front body prediction averages
        #res_ave = np.nanmedian(res[:3],axis=0)
        #print (" res_ave; ", res_ave.shape)
        
        res_ave = res[0]
        
        return res_ave, all_locs, idxc

    
        # 
    def generate_imputation_models(self):
        
        
        from sklearn.linear_model import BayesianRidge
        from sklearn.tree import DecisionTreeRegressor
        from sklearn.ensemble import ExtraTreesRegressor
        from sklearn.neighbors import KNeighborsRegressor
        
        self.model_type_names = ["BayesianRidge",
                                 "DecisionTreeRegressor",
                                 "ExtraTreesRegressor",
                                 "KNeighborsRegressor"]
        
        for animal_id in self.animal_ids:

            # 
            temp = np.vstack(self.cb.features_array[animal_id])
            print ("Raw Data: ", temp.shape)

            # 
            body_centres = [self.body_centre]
            
            for k in body_centres:

                fname_out = self.root_dir+"model_type"+str(self.model_type)+"_imputation_animal_id"+str(animal_id)+"_body_centre"+str(k)+".pckl"
                if os.path.exists(fname_out)==False:
                    
                    # centre the data on the particular feature being trained on
                    # TODO: try further aligning to 2 points; might give more stable results...
                    temp = temp-temp[:,k][:,None]

                    X_train = temp.reshape(temp.shape[0],-1)
                    print ("X_train: ", X_train.shape)

                    #
                    estimators = [BayesianRidge(),
                    DecisionTreeRegressor(max_features='sqrt', random_state=0),
                    ExtraTreesRegressor(n_estimators=10, random_state=0),
                    KNeighborsRegressor(n_neighbors=15)]

                    print ("fitting...animal id: ", animal_id, "  body centre: ", k)
                    imp = IterativeImputer(max_iter=10, 
                                           #n_nearest_features = 2,
                                           #sample_posterior = True,
                                           estimator=estimators[self.model_type],
                                           random_state=0)
                    imp.fit(X_train)
                    print ("done")

                    # 
                    with open(fname_out, "wb") as f:
                        pickle.dump(imp, f)

                        
#
root_dir = '/media/cat/1TB/dan/'
I = Impute(root_dir)
I.cb = cb
I.animal_id = 0
I.animal_ids = [I.animal_id]


# generate all required models; need clean data only
I.model_type = 2
I.body_centre = 0  # use head fixed only for now
I.generate_imputation_models()
I.load_models()


# test against ground truth/cleand data 
I.fname_dropout = '/home/cat/feats_dropout.tsv'
res, idx_drop = I.predict_imputation_ground_truth(drops=np.arange(3,6,1)) # drops = 'fixed' or None

# check missing features in the data
# I.calculate_missing_features()

# 
# I.plot_imputation_results(features_array, animal_id, idx_test, res, idx_drop)
# plt.suptitle("animal "+str(animal_id)+ " imputed vs. ground truth",fontsize=20)

# # 
# I.evaluate_imputation_error(features_array, animal_id, res, idx_train, idx_test)
# plt.suptitle("animal "+str(animal_id)+ "  Egocentric (fixed nose) errors (pixels)",fontsize=20)
# #plt.suptitle("animal "+str(animal_id)+ " NON-Egocentric (fixed nose) errors (pixels)",fontsize=20)


# plt.show()
print ("DONE")

Raw Data:  (218641, 6, 2)
(218641, 12)
... predting data ...
DONE


In [166]:
#####################################################
# print (res_ave.shape)
# print (all_locs.shape)
# print (idxc.shape, idxc)


def make_vae_data():
    import csv

    print (np.vstack(cb.features_array[0]).shape)

    X = np.vstack(cb.features_array[0])
    #X = X.reshape(X.shape[0],-1)

    max_drop_outs = 3
    n_frames = 50000
    n_frames = X.shape[0]

    # 
    with open('/home/cat/feats_ground_truth.tsv', 'w') as tsvfile:
        writer = csv.writer(tsvfile, delimiter='\t')
        #for record in SeqIO.parse("/home/fil/Desktop/420_2_03_074.fastq", "fastq"):
        #for k in trange(X.shape[0]): 
        for k in trange(n_frames): 

            temp = X[k]
            if False:
                idx = np.random.choice(np.arange(6), max_drop_outs, replace=False)

                if idx.shape[0]>0:
                    temp[idx]=np.nan
            temp = temp.reshape(-1)
            writer.writerow(temp)

    with open('/home/cat/feats_dropout.tsv', 'w') as tsvfile:
        writer = csv.writer(tsvfile, delimiter='\t')
        #for record in SeqIO.parse("/home/fil/Desktop/420_2_03_074.fastq", "fastq"):
        #for k in trange(X.shape[0]): 
        for k in trange(n_frames): 

            temp = X[k]
            if True:
                idx = np.random.choice(np.arange(6),max_drop_outs, replace=False)

                if idx.shape[0]>0:
                    temp[idx]=np.nan
            temp = temp.reshape(-1)
            writer.writerow(temp)        
        
def plot_multiple_imputation_results(frame_ids, I):
    all_locs = np.array(np.loadtxt('/home/cat/feats_ground_truth.tsv'))
    all_locs = all_locs.reshape(all_locs.shape[0],6,2)
    #all_locs = np.vstack(cb.features_array[0])
    print (all_locs.shape)

    #
    drops = np.arange(3,6,1)
    leftin = np.arange(0,3,1)

    #frame_ids = [13149,3399,32567,31647,14473,48186,39608,32802,46362,34761]
    fig=plt.figure()
    for k in range(10):
    #
        #id_ = np.random.choice(all_locs.shape[0],1)
        id_ = frame_ids[k]
        #print (idx_drop[:,id_].T)
    #     if np.array_equal(idx_drop[:,id_].T[0], drops):
    #         break

        # 
        ax=plt.subplot(2,5,k+1)

        ############## PLOT GROUND TRUTH #################
        plt.scatter(all_locs[id_,:,0]-100,
                    all_locs[id_,:,1],
                    s=np.arange(1,7,1)[::-1]*50,
                    c='black'
                   )

        plt.scatter(all_locs[id_,drops,0]-100,
                    all_locs[id_,drops,1],
                    s=np.arange(1,7,1)[::-1][drops]*50,
                    c='blue'
                   )    


        ############## PLOT IMPUTED #################
        plt.scatter(res[id_,:,0],
                   res[id_,:,1],
                    s=np.arange(1,7,1)[::-1]*50,
                    c='red'
                   )

        plt.scatter(res[id_,leftin,0],
                    res[id_,leftin,1],
                    s=(np.arange(7,0,-1)[leftin]*50),
                    c='black'
                   )
        plt.title(str(id_))

    #plt.suptitle(str(I.model_type_names[I.model_type]))
    plt.suptitle(I.model_type_names[I.model_type])

    
def plot_vae_scatter():

    #
    imputed = np.loadtxt('/home/cat/data_imputed.tsv')

    #
    gt = np.loadtxt('/home/cat/feats_ground_truth.tsv')
    gt_dropout = np.loadtxt('/home/cat/feats_dropout.tsv')

    #
    print (gt.shape)
    print (imputed.shape)
    print (gt_dropout.shape)

    frame_ids = []
    fig=plt.figure()
    for k in range(10):
        #
        while True:

            id_ = np.random.choice(np.arange(gt.shape[0]),1)

            # PLOT GT

            temp = gt_dropout[id_]
            #temp = temp.reshape()
           # print (temp)
            idx = np.where(np.isnan(temp))[1]
            #print (idx)
            
            # look only for the bottom 3 feature
            if np.array_equal(idx,np.arange(6,12,1))==False:
                continue


            temp = gt[id_]
            temp = temp.reshape(6,2)

            ax=plt.subplot(2,5,k+1)
            plt.scatter(temp[:,0]-100,
                       temp[:,1],
                       s=np.arange(1,7,1)[::-1]*50,
                       alpha=1,
                       c='blue')

            temp = gt_dropout[id_]
            temp = temp.reshape(6,2)
            plt.scatter(temp[:,0]-100,
                       temp[:,1],
                       s=np.arange(1,7,1)[::-1]*50,
                       alpha=1,
                       c='black')   

            # PLOT IMPUTATION
            temp5 = imputed[int(id_*5):int(id_*5+5)]
            print (temp5.shape)
            for t in range(temp5.shape[0]):
                temp = temp5[t]
                temp = temp.reshape(6,2)
                #print (temp.shape)
                plt.scatter(temp[:,0],
                       temp[:,1],
                       s=np.arange(1,7,1)[::-1]*50,
                       c='red',
                       alpha=.1)
            temp = np.mean(temp5,axis=0)
            temp = temp.reshape(6,2)
            plt.scatter(temp[:,0],
                   temp[:,1],
                   s=np.arange(1,7,1)[::-1]*50,
                   c='red',
                   alpha=1)        


            temp = gt_dropout[id_]
            temp = temp.reshape(6,2)
            plt.scatter(temp[:,0],
                       temp[:,1],
                       s=np.arange(1,7,1)[::-1]*50,
                       alpha=1,
                       c='black')   
            plt.title(str(id_))
            
            frame_ids.append(id_)
            break

    plt.suptitle("Variational Autoencoder with Arbitrary Conditioning (https://github.com/tigvarts/vaeac)")
    plt.show()
    return frame_ids    

#   
def evaluate_imputation_vae():

    # VAE ERROR
    imputed = np.loadtxt('/home/cat/data_imputed.tsv')
    gt = np.float32(np.loadtxt('/home/cat/feats_ground_truth.tsv'))
    
    gt = np.float32(gt.reshape(gt.shape[0],6,2))

    
    imputed = np.array(np.array_split(imputed, 5))
    imputed = np.mean(imputed,axis=0)
    
    imputed = imputed.reshape(imputed.shape[0],6,2)

    
    diff = np.float32(np.abs(gt-imputed))
    #print ("diff: ", diff.shape)

    errors = []
    for k in range(diff.shape[1]):
        errors.append([])

    for k in range(diff.shape[0]):
        for p in range(diff.shape[1]):
            temp = diff[k,p]
            #print (temp)
            tdiff = np.linalg.norm(temp)
            if tdiff>0:
                errors[p].append(tdiff)

    t =[]
    for k in range(len(errors)):
        temp = errors[k]
        pad = np.zeros(350000-len(errors[k]),'float32')+np.nan
        temp = np.concatenate((temp, pad))
        
        idx = np.where(temp<1E-8)[0]
        temp[idx]=np.nan
        t.append(temp)

    errors = np.float64(t).T

    columns = ['nose','spine1','spine2', 'spine3', 'spine4', 'spine5']
    df = pd.DataFrame(errors, columns = columns)

    print (df)

    fig=plt.figure()
    ax = sns.violinplot(data=df) #x=df['spine2'])
    plt.ylim(0,50)
    plt.title("VAE IMPUTATION")
    #plt.ylabel(" pixel error")

    return df
    
def evaluate_imputation_multivariate(I):

    # VAE ERROR
    imputed = I.res #np.loadtxt('/home/cat/data_imputed.tsv')
    gt = np.loadtxt('/home/cat/feats_ground_truth.tsv')
    
    gt = gt.reshape(gt.shape[0],6,2)

    print (imputed.shape)
    
#     imputed = np.array(np.array_split(imputed, 5))
#     print (imputed.shape)
#     imputed = np.mean(imputed,axis=0)
#     print (imputed.shape)
    
#     imputed = imputed.reshape(imputed.shape[0],6,2)
    #gt_dropout = np.loadtxt('/home/cat/feats_dropout.tsv')
    
    
#     temp = np.vstack(features_array[animal_id])
#     temp = temp-temp[:,0][:,None]


    diff = np.float32(np.abs(gt-imputed))
    #print ("diff: ", diff.shape)
    print ("Data type: ", type(diff[0][0][0]))

    errors = []
    for k in range(diff.shape[1]):
        errors.append([])

    for k in range(diff.shape[0]):
        for p in range(diff.shape[1]):
            temp = diff[k,p]
            #print (temp)
            tdiff = np.linalg.norm(temp)
            if tdiff>0:
                errors[p].append(tdiff)

    t =[]
    for k in range(len(errors)):
        temp = errors[k]
        pad = np.zeros(350000-len(errors[k]),'float32')+np.nan
        temp = np.concatenate((temp, pad))
        t.append(temp)

    data = np.array(t).T
    print ("DAT TIPE:", type(data[0][0]))
    columns = ['nose','spine1','spine2', 'spine3', 'spine4', 'spine5']
    df = pd.DataFrame(data, columns = columns)

    print (df)

    fig=plt.figure()
    
    ax = sns.violinplot(data=df) #x=df['spine2'])
    plt.ylim(0,50)
    plt.title(I.model_type_names[I.model_type])

    return df
#     ax=plt.subplot(2,1,2)
#     plt.title("Zoom",fontsize=20)
#     sns.violinplot(data=df) #x=df['spine2'])
#     plt.ylabel(" pixel error")
#     plt.ylim(0,50)


In [167]:
# # 
# make_vae_data()

In [168]:
df1 = evaluate_imputation_vae()

#df2 = evaluate_imputation_multivariate(I)



/home/cat/anaconda3/envs/gerbil/lib/python3.7/site-packages/ipykernel_launcher.py:232: RuntimeWarning: invalid value encountered in less


        nose    spine1     spine2     spine3     spine4     spine5
0        NaN  6.915198  14.171872  32.746586  13.954534  15.646737
1        NaN  6.920685  14.155935  32.661476  13.741578  15.489763
2        NaN  6.928537  13.574489  32.219589  13.784212  17.295628
3        NaN  6.799797  13.708711  32.038719  12.180178  15.949262
4        NaN  6.874421  13.581362  32.197235  12.272676  15.428333
...      ...       ...        ...        ...        ...        ...
349995   NaN       NaN        NaN        NaN        NaN        NaN
349996   NaN       NaN        NaN        NaN        NaN        NaN
349997   NaN       NaN        NaN        NaN        NaN        NaN
349998   NaN       NaN        NaN        NaN        NaN        NaN
349999   NaN       NaN        NaN        NaN        NaN        NaN

[350000 rows x 6 columns]


In [144]:


# 
frame_ids = plot_vae_scatter()
print ("frame ids: ", frame_ids)  
    
#        
plot_multiple_imputation_results(frame_ids, I)


(50000, 12)
(250000, 12)
(50000, 12)
(5, 12)
(5, 12)
(5, 12)
(5, 12)
(5, 12)
(5, 12)
(5, 12)
(5, 12)
(5, 12)
(5, 12)
frame ids:  [array([18594]), array([42767]), array([44510]), array([11645]), array([26048]), array([27577]), array([15465]), array([30459]), array([43248]), array([36295])]
(50000, 6, 2)


In [ ]:
##########################################################
##########################################################
##########################################################
from scipy.spatial import cKDTree
import joblib

def knn_triage(th, pca_wf):
    tree = cKDTree(pca_wf)
    dist, ind = tree.query(pca_wf, k=6)
    dist = np.sum(dist, 1)
    idx_keep1 = dist <= np.percentile(dist, th)
    return idx_keep1



# Fit the PCA object, but do not transform the data
for k in range(4):
    ax=plt.subplot(2,2,k+1)
    
    temp = features_array[k]
    d = []
    clrs = []
    for p in range(len(temp)):
        d.append(temp[p])
        clrs.extend(np.zeros(temp[p].shape[0])+p)
    
    clrs = np.array(clrs)
    d = np.vstack(d)
    print ("D: ", d.shape)
    d = d.reshape(d.shape[0],-1)
    continue
    #d = sklearn.preprocessing.normalize(d)

    # remove 1% of outliers
    if True:
        th = 95  # % of data to keep
        idx_keep = knn_triage(th, d)
        print (" d before traige: ", d.shape)
        d = d[idx_keep]
        print (" d after traige: ", d.shape)
        clrs = clrs[idx_keep]
    
    
    if False:
        pca = PCA(2)

        print ("... data into pca: ", d.shape)

        feats_pca = pca.fit_transform(d)
        print (feats_pca.shape)

        # 
        plt.scatter(feats_pca[::5,0],
           feats_pca[::5,1],
            #c=np.arange(feats_pca.shape[0])[::5],
            c=clrs[::5],
            alpha=.05)
        
    if True:
        
#         import gpumap
#         #from sklearn.datasets import load_digits

#         #digits = load_digits()
#         print ("Data into gpumap: ", d.shape)
#         feats_pca = gpumap.GPUMAP().fit_transform(d)
#         print ("Data out of gpumap: ", feats_pca.shape)

        import umap
    
        umap = umap.UMAP(n_components=2,
                        init='random',
                        random_state=0)
        
        d = d[::2]
        clrs = clrs[::2]
        
        print ("... data into umap: ", d.shape)
        
        if False:
            umap_ = umap.fit(d) #[::10])
            feats_pca = umap_.transform(d)
        else:
            feats_pca = umap.fit_transform(d) #[::10])
        
        
            # remove 1% of outliers
        if True:
            th = 90  # % of data to keep
            idx_keep = knn_triage(th, feats_pca)
            print (" d before traige: ", feats_pca.shape)
            feats_pca = feats_pca[idx_keep]
            print (" d after traige: ", feats_pca.shape)
            clrs = clrs[idx_keep]
        
        plt.scatter(feats_pca[:,0],
               feats_pca[:,1],
                #c=np.arange(feats_pca.shape[0])[::5],
                c=clrs,
                alpha=.05)
    if False:
        
        #from openTSNE import TSNE
        #print ("... data into tsne: ", d.shape)
        #feats_pca = TSNE().fit(d)
        
        
        from fastTSNE import TSNE

        tsne = TSNE(
            n_components=2, perplexity=30, learning_rate=100, early_exaggeration=12,
            n_jobs=4, 
            #angle=0.5, 
            initialization='random', metric='euclidean',
            n_iter=750, early_exaggeration_iter=250, neighbors='exact',
            negative_gradient_method='bh', min_num_intervals=10,
            #ints_in_inverval=2, 
            #late_exaggeration_iter=100, 
            #late_exaggeration=4,
        )
        
        # 
        feats_pca = tsne.fit(d)

        print (" output: ", feats_pca.shape)


        plt.scatter(feats_pca[:,0],
            feats_pca[:,1],
            #c=np.arange(feats_pca.shape[0])[::5],
            c=clrs,
            alpha=.05)

    # 
    plt.title("Animal:"+str(k))
    
    
plt.suptitle("Static vertically aligned postures",fontsize=20)
plt.show()
#plt.show()

In [15]:
lens = [218641, 94647, 132861, 176982]

lens = np.array(lens)
print (np.round(lens/(23*89900)*100,2))

[10.57  4.58  6.43  8.56]


In [44]:
#################################################
############### IMPUTE MISSING DATA #############
#################################################


from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
imp = IterativeImputer(max_iter=10, random_state=0)
imp.fit([[1, 2], [3, 6], [4, 8], [np.nan, 3], [7, np.nan]])
X_test = [[np.nan, 2], [6, np.nan], [np.nan, 6]]
# the model learns that the second feature is double the first

print(np.round(imp.transform(X_test)))

[[ 1.  2.]
 [ 6. 12.]
 [ 3.  6.]]


In [ ]:
##############################################
########## FEATURIZE BEHAVIOR CHUNKS #########
##############################################
from sklearn import decomposition
import sklearn

fig = plt.figure()
X_all = []
n_events = []
for animal_id in animal_ids:
    X = X4[animal_id].copy()
    X = X.reshape(X.shape[0], -1)
    print (X.shape)
    X_all.append(X)
    n_events.append(X.shape[0])

#     
X_all = np.vstack(X_all)
print (X_all.shape)
X = sklearn.preprocessing.normalize(X_all)

#
if True:
    pca = decomposition.PCA(n_components=3)

    X_pca = pca.fit_transform(X_all)
    print (X_pca.shape)
    
if False:
    import umap
    umap = umap.UMAP(n_components=2,
                    init='random',
                    random_state=0)

    umap_ = umap.fit(X_all[::10])

    X_pca = umap_.transform(X_all)
        

print ("plotting: ", X_pca.shape)


print (n_events)
fig=plt.figure()
for k in range(4):
    ax = plt.subplot(2,2,k+1)
    start = np.int32(n_events[:k]).sum()
    end = np.int32(n_events[:k+1]).sum()
    print (start, end)
    plt.scatter(X_pca[start:end,0],
                X_pca[start:end,1],
               alpha=.1)

plt.show()